In [ ]:
import pandas as pd
import numpy as np
import requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* define function to load a company list and filter for **founding year**, **industry** and **size**

In [ ]:
def load_companies(path, filter_year=None, filter_industry=None, filter_size=None, n=None):
    df = pd.read_json(path)
    if filter_year:
        f = (str(x) for x in filter_year)
        df = df.loc[df['founded'].isin(f)]
    
    if filter_industry:
        df = df.loc[df['industry'].isin(filter_industry)]
    
    if filter_size:
        size_dict = {10:'1-10',
                     50:'11-50',
                     200:'51-200', 
                     500:'201-500', 
                     1000:'501-1000', 
                     5000:'1001-5000',
                     10000:'5001-10000', 
                     10001:'10001+'
                    }
        f = [size_dict[x] for x in filter_size]
        df = df.loc[df['size'].isin(f)]
    
    if n:
        df = df[:n]
    
    return df.reset_index(drop=True), np.array(df['website'])

* load company dataframe and company list
* needs dataset created from https://www.peopledatalabs.com/company-dataset, filtered with MySql such that **only CA companies and only tech**)
* additional filtering such that **industry** = 'internet'


In [ ]:
#company_df, company_website_list = load_companies('../data/company_list_CA.json', 
company_df, company_website_list = load_companies('/content/drive/MyDrive/Accountstory/data/company_list_CA.json',
                            #filter_year=[2019], 
                            filter_industry=['internet'], 
                            #filter_size=[500,
                            #             1000]
                            n = 1000
                           )
company_df[100:200]

* define function to load non-DMs by calling the peopledatalabs.com person search api

In [ ]:
 def load_data(API_KEY, company_website_list ,n):

    PDL_URL = "https://api.peopledatalabs.com/v5/person/search"

    H = {
      'Content-Type': "application/json",
      'X-api-key': API_KEY
    }
    

    COMPANY_DOMAINS_STRING_REP = ", ".join(
      (f"'{site}'" for site in company_website_list)
    )

    SQL_QUERY = \
    f"""
      SELECT * FROM person
      WHERE job_company_website IN ({COMPANY_DOMAINS_STRING_REP})
      AND job_company_location_region IN ('california');
    """

    P = {
      'sql': SQL_QUERY,
      'size': n
    }

    response = requests.get(
      PDL_URL,
      headers=H,
      params=P
    ).json()

    if response["status"] == 200:
        return response['data']

        print(f"successfully grabbed {len(response['data'])} records from PDL")
        print(f"{response['total']} total PDL records exist matching this query")
    else:
        print("NOTE. The eager beaver was not so eager. See error and try again.")
        print("error:", response)

In [ ]:
key = # peopledatalabs API key

# API allows only 100 results per call. Call it several times to get more results.
df = pd.DataFrame(load_data(company_website_list[150:250], 100))
for i in range(3,10):
  df = df.append(pd.DataFrame(load_data(company_website_list[50+(i*100):150+(i*100)], 100)))


In [ ]:
df.drop('index',axis=1, inplace=True)

* export **people_list_nDM.json**

In [ ]:
df.to_json('/content/drive/MyDrive/Accountstory/data/people_list_nDM_20210930.json')